In [1]:
from letta_client import Letta
from letta_client.types.child_tool_rule import ChildToolRule

client = Letta(base_url="http://localhost:8283")

In [2]:
all_mcp = client.tools.list_mcp_servers()

In [3]:
mcp_server_name = "toolbox"
mcp_tools = client.tools.list_mcp_tools_by_server(
    mcp_server_name=mcp_server_name,
)

In [4]:
tool_ids = []

for tool in mcp_tools:
    added = client.tools.add_mcp_tool(
        mcp_server_name=mcp_server_name,
        mcp_tool_name=tool.name
    )
    tool_ids.append(added.id)

In [5]:
agent_state = client.agents.create(
    name="universal_mcp_assistant",
    system=(
        "You are an assistant that can perform any day-to-day task by routing through MCP.\n"
        "Whenever the user asks for something (e.g., “get me the latest information on a certain topic”, “check for any important new emails”, "
        "“find new job postings I am qualified for”), you should:\n"
        "  1. Call `search_servers` with a natural-language query describing the task, and set `n` to 1 so you only retrieve one result.\n"
        "     • For example: { \"name\": \"search_servers\", \"arguments\": { \"query\": \"Find MCP server for checking unread emails\", \"n\": 1 }}\n"
        "     • The single returned MCP’s qualifiedName will expose exactly the tool you need.\n"
        "  2. From that MCP’s qualifiedName, pick the exact sub-tool name (e.g., ‘check_email’) and call `use_tool` with:\n"
        "     {\n"
        "       \"qualifiedName\": <the server you got from search_servers>,\n"
        "       \"parameters\": {\n"
        "         \"name\": <the sub-tool to invoke>,\n"
        "         \"arguments\": { … }       # whatever arguments that sub-tool requires\n"
        "       }\n"
        "     }\n"
        "  3. If the `use_tool` call returns an error containing a `configUrl`, that means the MCP tool requires configuration.\n"
        "     • Prompt the user with a message like: “This tool needs to be configured before use. Please configure it here: <configUrl>.”\n"
        "     • Do not treat it as a failure—simply provide the link so the user can complete configuration.\n"
        "  4. Once the `use_tool` call succeeds, take the JSON result and use it to:\n"
        "     • Answer whatever the user originally asked, or\n"
        "     • Confirm that the requested action has been completed successfully.\n"
        "If you see any non-unicode characters when using a tool, please ignore them so that you don't crash on stdout."
        "Repeat this flow for every new user request."
    ),
    model="openai/gpt-4o",
    embedding="openai/text-embedding-3-small",
    tool_ids=tool_ids,
    tool_rules=[
        ChildToolRule(tool_name="use_tool", children=["send_message"]),
    ]
)

In [6]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Get me the weather in Atlanta."
        },
    ],
)

In [7]:
for msg in response.messages:
    if msg.message_type == "assistant_message":
        print(msg.content)

The weather in Atlanta is mainly clear with a temperature of 28.7°C. The relative humidity at 2 meters is 44%, and the dew point temperature at 2 meters is 15.3°C.


In [8]:
response = client.agents.messages.create(
    agent_id=agent_state.id,
    messages=[
        {
            "role": "user",
            "content": "Can you get me some nice youtube recommendations."
        },
    ],
)

In [9]:
for msg in response.messages:
    if msg.message_type == "assistant_message":
        print(msg.content)

This YouTube tool needs to be configured before use. Please configure it here: [YouTube MCP Server Configuration](https://smithery.ai/server/@coyaSONG/youtube-mcp-server/config).
